In [67]:
import string
import pandas as pd
import numpy as np
import requests
import unidecode
import re

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"

try:
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)

except:
    print(f"Erreur de chargement du lien - {url}")
    quit()


liste_salaires = df['lieu'].apply(lambda x : x[1] if len(x) > 1 else '')

df['lieu'] = df['lieu'].apply(lambda x : x[0])

df['Nom de la société'] = df['Type de poste'].apply(lambda x : x[2] if len(x) > 2 else '')
df['Type de contrat'] = df['Type de poste'].apply(lambda x : x[7].split(' - ')[0] if len(x) > 7 else '')
df.drop(columns='Type de poste', inplace=True)

df = df.apply(lambda c : c.apply(lambda x : ','.join([s.strip("\n ").lower() for s in (x.split(',') if type(x) != list else x)])))

liste_dates = df['Date de publication'].apply(lambda x: '0 j' if x.find('heures') != -1 else x.removeprefix('postée il y a ').replace('postée hier', '1 j').strip('iours'))

df['Date de publication'] = pd.to_datetime(liste_dates.apply(
    lambda x :
        pd.to_datetime("2023-01-15") - pd.DateOffset(months=int(x.split()[0])) if x.split()[1] == 'm'
        else pd.to_datetime("2023-01-15") - pd.DateOffset(days=int(x.split()[0]))
    ))

df['Salaire minimum'] = liste_salaires.apply(lambda x : float(x.split(' - ')[0].strip(' €\n').replace('.', '').replace(',', '.')) if x != '' else np.nan)
df['Salaire maximum'] = liste_salaires.apply(lambda x : float(x.split(' - ')[1].strip(' €/an\n' + string.ascii_letters).replace('.', '').replace(',', '.')) if x != '' else np.nan)

noms_ville = ['la defense', 'guyancourt', 'antony', 'paris']

df['lieu'] = df['lieu'].apply(
    lambda x : (
        [n for n in noms_ville for i in unidecode.unidecode(x).replace(' - ', ',').replace('-', ' ').split(',') if n in i] +
        [unidecode.unidecode(x).replace('-', ' ')]
        )[0]
    )

#remplace si data analyst dans vavlue par data analyste etc
terms_to_replace = {'analyst': 'data analyst', 'datascientist': 'data scientist', 'data engineer': 'ingénieur','conultant/analyste': 'consultant/analyste','2018788': '',
                    'developer': 'developpeur','full stacke': 'developpeur','global technical seo manager': 'manager','engineer': 'ingenieur','lead': 'chef','full': 'developpeur','ingénieur': 'ingenieur','analyste': 'data analyst','product': 'chef de projet'}
df["Intitulé du poste"] = df["Intitulé du poste"].apply(lambda x: ' '.join([terms_to_replace.get(term, term) for term in unidecode.unidecode(x).split()]))

keywords = ["stage","business analyst","analyste fonctionnel","architecte","apprenti","data ingenieur","data ingénieur", "data engineer","data analyst","data scientist","consultant","ingenieur","chef de projet","concepteur","alternance",
            "technical leader","technicien","responsable","référent","expert","developpeur","specialiste","referent","manager","postdoctorant","internship"]

for keyword in keywords:
    df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(rf"(.*{keyword})", keyword, regex=True)
    df["Intitulé du poste"] = df["Intitulé du poste"].str.replace(rf"({keyword}.*)", keyword, regex=True)

df['Intitulé du poste'] = df['Intitulé du poste'].apply(lambda x: re.sub(r'[hf]/[hf]|[-\/()]', '', x))

df.to_csv("data_test1.csv")

Erreur de chargement du lien - https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json


KeyError: 'Type de poste'

: 

In [6]:
print(df["Salaire maximum"].count())

43


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Séparer les données en variables indépendantes (X) et dépendante (y)
X = df[['Intitulé du poste', 'Date de publication', 'lieu', 'competences','Type de contrat']]
y = df[['Salaire minimum', 'Salaire maximum']]

# Séparer les données en jeux d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Créer un objet de régression linéaire
reg = LinearRegression()

# Entraîner le modèle sur les données d'entraînement
reg.fit(X_train, y_train)

# Effectuer des prédictions sur les données de test
y_pred = reg.predict(X_test)

# Afficher les résultats de précision
print(reg.score(X_test, y_test))